In [25]:
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import torch.utils.data as Data
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
import time
import multiprocessing as mp
from os import walk
feat_num = 9

In [2]:
time_list = []
path = "/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/"
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        time_list.append(j)
time_list.sort()     
time_list

100%|██████████| 26/26 [00:00<00:00, 166746.03it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [57]:
trainx , trainy , testx , testy = [] , [] , [],  []

for train in tqdm(time_list[-7:-2]):
    trainx.append(pd.read_parquet(path+ "Pooling_Norm_Daily/" + train).set_index(["timestamp","ticker"]))
    trainy.append(pd.read_parquet(data_path + train).set_index(["timestamp","ticker"])['target'])
trainx = pd.concat(trainx,axis=0)
trainy = pd.concat(trainy,axis=0)

display(trainx)
display(trainy)

for test in tqdm(time_list[-2:-1]):
    testx.append(pd.read_parquet(path+ "Pooling_Norm_Daily/" + test).set_index(["timestamp","ticker"]))
    testy.append(pd.read_parquet(data_path + test).set_index(["timestamp","ticker"])['target'])
testx = pd.concat(testx,axis=0)
testy = pd.concat(testy,axis=0)

display(testx)
display(testy)

100%|██████████| 5/5 [00:07<00:00,  1.44s/it]


0         1         2         3          4  \
timestamp           ticker                                                      
2019-09-02 09:30:00 1       0.678544  0.911029  0.923079  0.819949  19.449303   
                    2       0.505607  0.469982  0.506814  0.518341   0.522568   
                    4      -0.034000 -0.033118 -0.033583 -0.033979  -0.107561   
                    5      -0.034001 -0.033122 -0.033616 -0.033980  -0.106927   
                    6      -0.033789 -0.032978 -0.033486 -0.033776  -0.017472   
...                              ...       ...       ...       ...        ...   
2020-12-31 09:30:00 688160 -0.037991 -0.040224 -0.039223 -0.038744  -0.101546   
                    3019   -0.039213 -0.041410 -0.040196 -0.039836  -0.105911   
                    300907 -0.038737 -0.040866 -0.039739 -0.039371  -0.100740   
                    300909 -0.038613 -0.040648 -0.039745 -0.039256  -0.101858   
                    688219 -0.038884 -0.041166 -0.040038 -0.039566  -0.091742   

                                   5         6         7         8         9  \
timestamp           ticker                                                     
2019-09-02 09:30:00 1       0.037095  0.481085  1.045546  0.887051  0.896379   
                    2       0.950653 -0.108248 -0.092828  0.483910  0.397593   
                    4      -0.093072 -0.153431 -0.154710 -0.034126 -0.032681   
                    5      -0.022473 -0.154583 -0.155497 -0.034136 -0.032684   
                    6      -0.055151 -0.089717 -0.086429 -0.034027 -0.032583   
...                              ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688160 -0.066243  0.642861  0.443365 -0.040209 -0.038461   
                    3019    0.205344  0.057019 -0.001573 -0.041529 -0.039543   
                    300907  0.029983  0.577627  0.393821 -0.040815 -0.038835   
                    300909 -0.071384  0.453229  0.299343 -0.040595 -0.039008   
                    688219  0.018562  0.646257  0.445944 -0.041220 -0.039355   

                            ...       314       315       316       317  \
timestamp           ticker  ...                                           
2019-09-02 09:30:00 1       ... -0.787435  2.159639  2.114878  2.178351   
                    2       ...  0.014650  5.649766  5.524472  5.692661   
                    4       ...  0.204913 -0.017689 -0.016078 -0.018281   
                    5       ... -0.068107 -0.094460 -0.095006 -0.094247   
                    6       ...  0.512949  0.144878  0.144651  0.146115   
...                         ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688160  ... -0.023950 -0.096209 -0.096343 -0.096425   
                    3019    ...  1.097135 -0.114166 -0.114927 -0.113697   
                    300907  ...  0.102655 -0.102497 -0.102636 -0.102364   
                    300909  ... -0.012331 -0.076979 -0.077426 -0.077423   
                    688219  ... -0.416078 -0.129001 -0.129640 -0.128561   

                                 318       319       320       321       322  \
timestamp           ticker                                                     
2019-09-02 09:30:00 1       2.141479  2.140690  4.270585  0.256924 -0.467431   
                    2       5.578581  5.585099  2.069260 -0.306779 -0.565508   
                    4      -0.018024 -0.017490 -0.501185  0.724720 -0.283452   
                    5      -0.094707 -0.094644 -0.175399  0.099346 -0.348100   
                    6       0.144825  0.144901 -0.135776  0.199812 -0.362439   
...                              ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688160 -0.096885 -0.096629 -0.420126 -3.476146  1.638636   
                    3019   -0.114365 -0.114259 -0.374179 -0.104727  2.284834   
                    300907 -0.102976 -0.102502 -0.330478 -1.104214  6.082891   
                    300909 -0.078097 -0.077509 -0.338606 -1.205655  4.976046   
    

timestamp            ticker
2019-09-02 09:30:00  1         0.058657
2019-09-03 09:30:00  1         0.008287
2019-09-04 09:30:00  1         0.019553
2019-09-05 09:30:00  1         0.006868
2019-09-06 09:30:00  1        -0.000680
                                 ...   
2020-12-31 09:30:00  300907   -0.080864
2020-12-30 09:30:00  300909   -0.014117
2020-12-31 09:30:00  300909   -0.088868
2020-12-30 09:30:00  688219   -0.074826
2020-12-31 09:30:00  688219   -0.104478
Name: target, Length: 1235213, dtype: float64

100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


0         1         2         3         4  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       0.407905  0.289338  0.466350  0.383819  1.802049   
                    2       0.326919  0.263456  0.295429  0.290163  2.967022   
                    4      -0.049580 -0.048750 -0.048699 -0.048818 -0.083488   
                    5      -0.051207 -0.050737 -0.050329 -0.050548 -0.099076   
                    6      -0.051122 -0.050579 -0.050227 -0.050441 -0.097335   
...                              ...       ...       ...       ...       ...   
2021-02-26 09:30:00 300928 -0.062378 -0.061238 -0.063163 -0.061787 -0.105753   
                    605277 -0.062455 -0.061291 -0.063189 -0.061839 -0.125982   
                    300926 -0.062411 -0.061253 -0.063175 -0.061801 -0.123635   
                    300927 -0.062380 -0.061270 -0.062965 -0.061785 -0.123941   
                    688617 -0.048909 -0.051899 -0.054084 -0.052506 -0.091580   

                                   5         6         7         8         9  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1      -1.413486 -0.055329 -0.000242  0.432748  0.443720   
                    2      -0.733268  0.285176  0.269749  0.335488  0.263364   
                    4      -0.046416  0.189171  0.217015 -0.050578 -0.049983   
                    5       0.010590 -0.157427 -0.153427 -0.052331 -0.051327   
                    6      -0.051407 -0.154564 -0.150423 -0.052223 -0.051247   
...                              ...       ...       ...       ...       ...   
2021-02-26 09:30:00 300928 -0.154694  1.303759  0.880051 -0.062767 -0.063237   
                    605277  0.181610 -0.116102 -0.140585 -0.062796 -0.063258   
                    300926 -0.194953 -0.034493 -0.090735 -0.062782 -0.063243   
                    300927 -0.157067  2.585192  1.566905 -0.062608 -0.062969   
                    688617 -0.214781  3.561969  2.510574 -0.055064 -0.056654   

                            ...       314       315       316       317  \
timestamp           ticker  ...                                           
2021-01-04 09:30:00 1       ...  0.150124  2.855153  2.851216  2.849224   
                    2       ...  1.029937  6.244212  6.206035  6.252585   
                    4       ... -2.204087 -0.028681 -0.029197 -0.029784   
                    5       ...  0.814870 -0.122606 -0.123642 -0.121710   
                    6       ...  1.033170  0.134660  0.133595  0.134522   
...                         ...       ...       ...       ...       ...   
2021-02-26 09:30:00 300928  ... -1.203937 -0.093495 -0.093817 -0.093034   
                    605277  ... -0.568329 -0.129967 -0.130320 -0.129027   
                    300926  ... -1.227811 -0.129994 -0.129839 -0.129482   
                    300927  ...  0.095596 -0.112089 -0.110811 -0.112446   
                    688617  ... -0.545019  0.166539  0.168919  0.160796   

                                 318       319       320       321        322  \
timestamp           ticker                                                      
2021-01-04 09:30:00 1       2.865968  2.853820  2.888370  1.123278  -0.634337   
                    2       6.240249  6.232339  1.568911  0.898320  -0.587184   
                    4      -0.030253 -0.029623 -0.325135 -2.139031   0.755537   
                    5      -0.122688 -0.122698 -0.192109  0.583967  -0.576121   
                    6       0.134658  0.133882 -0.176615  1.190075  -0.566626   
...                              ...       ...       ...       ...        ...   
2021-02-26 09:30:00 300928 -0.093413 -0.093318 -0.292259 -1.688301   3.526100   
                    605277 -0.129584 -0.129622 -0.277359 -1.939531   2.572125   
                    300926 -0.129594 -0.129564 -0.190535 -0.939824   7.283010   
                    300927 -0.111119 -0.111399 -0.189203 -1.281072  12.435535   
     

timestamp            ticker
2021-01-04 09:30:00  1         0.047120
2021-01-05 09:30:00  1         0.108152
2021-01-06 09:30:00  1         0.161504
2021-01-07 09:30:00  1         0.059426
2021-01-08 09:30:00  1         0.055276
                                 ...   
2021-02-26 09:30:00  300926   -0.000995
2021-02-25 09:30:00  300927   -0.080202
2021-02-26 09:30:00  300927   -0.000309
2021-02-25 09:30:00  688617   -0.009645
2021-02-26 09:30:00  688617   -0.065900
Name: target, Length: 143349, dtype: float64

In [47]:
train = pd.concat([trainx,trainy],axis=1)
train.dropna(inplace=True)
trainx = train.drop("target",axis=1)
trainy = train['target']
del train
display(trainx)
display(trainy)

0         1         2         3          4  \
timestamp           ticker                                                      
2019-09-02 09:30:00 1       0.678544  0.911029  0.923079  0.819949  19.449303   
                    2       0.505607  0.469982  0.506814  0.518341   0.522568   
                    4      -0.034000 -0.033118 -0.033583 -0.033979  -0.107561   
                    5      -0.034001 -0.033122 -0.033616 -0.033980  -0.106927   
                    6      -0.033789 -0.032978 -0.033486 -0.033776  -0.017472   
...                              ...       ...       ...       ...        ...   
2020-12-31 09:30:00 688598  0.147733  0.081896  0.165457  0.132351  -0.012713   
                    688599 -0.039287 -0.041514 -0.040303 -0.039937  -0.119215   
                    688600 -0.039352 -0.041643 -0.040369 -0.040017  -0.109832   
                    688788  0.047678  0.038114  0.071518  0.054767  -0.110674   
                    688981 -0.039327 -0.041594 -0.040345 -0.039984  -0.112541   

                                   5         6         7         8         9  \
timestamp           ticker                                                     
2019-09-02 09:30:00 1       0.037095  0.481085  1.045546  0.887051  0.896379   
                    2       0.950653 -0.108248 -0.092828  0.483910  0.397593   
                    4      -0.093072 -0.153431 -0.154710 -0.034126 -0.032681   
                    5      -0.022473 -0.154583 -0.155497 -0.034136 -0.032684   
                    6      -0.055151 -0.089717 -0.086429 -0.034027 -0.032583   
...                              ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688598  0.366231  6.683849  3.799423  0.155260  0.157075   
                    688599 -0.071455 -0.156544 -0.175599 -0.041601 -0.039611   
                    688600  0.007144 -0.173326 -0.176453 -0.041692 -0.039665   
                    688788  0.582867 -0.268458 -0.248767  0.069342  0.047980   
                    688981 -0.002485 -0.178356 -0.180336 -0.041657 -0.039643   

                            ...       315       316       317       318  \
timestamp           ticker  ...                                           
2019-09-02 09:30:00 1       ...  2.159639  2.114878  2.178351  2.141479   
                    2       ...  5.649766  5.524472  5.692661  5.578581   
                    4       ... -0.017689 -0.016078 -0.018281 -0.018024   
                    5       ... -0.094460 -0.095006 -0.094247 -0.094707   
                    6       ...  0.144878  0.144651  0.146115  0.144825   
...                         ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688598  ...  0.009165  0.010370  0.009084  0.010031   
                    688599  ... -0.125045 -0.124420 -0.125307 -0.124879   
                    688600  ... -0.133483 -0.134376 -0.132840 -0.133644   
                    688788  ...  0.088794  0.091331  0.076302  0.078879   
                    688981  ... -0.080413 -0.079790 -0.079569 -0.079450   

                                 319       320       321       322       323  \
timestamp           ticker                                                     
2019-09-02 09:30:00 1       2.140690  4.270585  0.256924 -0.467431 -0.356665   
                    2       5.585099  2.069260 -0.306779 -0.565508 -0.630599   
                    4      -0.017490 -0.501185  0.724720 -0.283452 -0.283844   
                    5      -0.094644 -0.175399  0.099346 -0.348100 -0.470585   
                    6       0.144901 -0.135776  0.199812 -0.362439 -0.454678   
...                              ...       ...       ...       ...       ...   
2020-12-31 09:30:00 688598  0.009882 -0.431090  2.397720  0.763765  0.315498   
                    688599 -0.124644  0.951751 -0.296562  4.714858  3.544533   
                    688600 -0.133636 -0.489636  0.209628 -0.226862 -0.210631   
                    688788  0.082542 -0.451811 -5.526516  2.325200  1.591583   
    

In [58]:
test = pd.concat([testx,testy],axis=1)
test.dropna(inplace=True)
testx = test.drop("target",axis=1)
testy = test['target']
index = testy.index
del test
display(testx)
display(testy)

0         1         2         3         4  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       0.407905  0.289338  0.466350  0.383819  1.802049   
                    2       0.326919  0.263456  0.295429  0.290163  2.967022   
                    4      -0.049580 -0.048750 -0.048699 -0.048818 -0.083488   
                    5      -0.051207 -0.050737 -0.050329 -0.050548 -0.099076   
                    6      -0.051122 -0.050579 -0.050227 -0.050441 -0.097335   
...                              ...       ...       ...       ...       ...   
2021-02-26 09:30:00 688698 -0.062484 -0.061330 -0.063185 -0.061871 -0.112251   
                    688699 -0.062030 -0.060865 -0.063086 -0.061416 -0.120970   
                    688777 -0.060251 -0.059094 -0.061170 -0.059643 -0.108395   
                    688788 -0.062158 -0.061053 -0.062676 -0.061596 -0.126771   
                    688981 -0.062385 -0.061248 -0.063007 -0.061785 -0.093783   

                                   5         6         7         8         9  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1      -1.413486 -0.055329 -0.000242  0.432748  0.443720   
                    2      -0.733268  0.285176  0.269749  0.335488  0.263364   
                    4      -0.046416  0.189171  0.217015 -0.050578 -0.049983   
                    5       0.010590 -0.157427 -0.153427 -0.052331 -0.051327   
                    6      -0.051407 -0.154564 -0.150423 -0.052223 -0.051247   
...                              ...       ...       ...       ...       ...   
2021-02-26 09:30:00 688698  0.036640  0.345962  0.188482 -0.062795 -0.063257   
                    688699 -0.055091  0.216848  0.095257 -0.062708 -0.063167   
                    688777 -0.022276  0.616240  0.306764 -0.062306 -0.063154   
                    688788 -0.235972 -0.102793 -0.132456 -0.062298 -0.062800   
                    688981 -0.047337 -0.156038 -0.164980 -0.062611 -0.063065   

                            ...       314       315       316       317  \
timestamp           ticker  ...                                           
2021-01-04 09:30:00 1       ...  0.150124  2.855153  2.851216  2.849224   
                    2       ...  1.029937  6.244212  6.206035  6.252585   
                    4       ... -2.204087 -0.028681 -0.029197 -0.029784   
                    5       ...  0.814870 -0.122606 -0.123642 -0.121710   
                    6       ...  1.033170  0.134660  0.133595  0.134522   
...                         ...       ...       ...       ...       ...   
2021-02-26 09:30:00 688698  ... -0.649731 -0.138908 -0.139269 -0.138143   
                    688699  ... -1.687068 -0.071529 -0.071814 -0.070972   
                    688777  ...  0.618691 -0.035834 -0.035809 -0.035736   
                    688788  ... -1.749871  0.020856  0.024184  0.019507   
                    688981  ... -1.533473 -0.076440 -0.076588 -0.075338   

                                 318       319       320       321       322  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       2.865968  2.853820  2.888370  1.123278 -0.634337   
                    2       6.240249  6.232339  1.568911  0.898320 -0.587184   
                    4      -0.030253 -0.029623 -0.325135 -2.139031  0.755537   
                    5      -0.122688 -0.122698 -0.192109  0.583967 -0.576121   
                    6       0.134658  0.133882 -0.176615  1.190075 -0.566626   
...                              ...       ...       ...       ...       ...   
2021-02-26 09:30:00 688698 -0.138551 -0.138552 -0.301666 -0.901053  1.039237   
                    688699 -0.071314 -0.071292 -0.312344  0.206770  2.148129   
                    688777 -0.034809 -0.035707 -0.307346 -0.400252  1.156100   
                    688788  0.020596  0.021923 -0.359184  0.151718  0.754048   
                 

timestamp            ticker
2021-01-04 09:30:00  1         0.047120
                     2         0.038732
                     4        -0.078782
                     5        -0.063241
                     6        -0.074468
                                 ...   
2021-02-26 09:30:00  688698    0.087517
                     688699    0.124138
                     688777    0.033333
                     688788   -0.063957
                     688981    0.051677
Name: target, Length: 143348, dtype: float64

In [52]:
trainx = torch.from_numpy(np.array(trainx))
trainy = torch.from_numpy(np.array(trainy).reshape(-1,1))
testx = torch.from_numpy(np.array(testx))
testy = torch.from_numpy(np.array(testy).reshape(-1,1))
print("trainx.shape: " , trainx.shape)
print("trainy.shape: " , trainy.shape)
print("testx.shape: " , testx.shape)
print("testy.shape: " , testy.shape)

trainx.shape:  torch.Size([1235192, 324])
trainy.shape:  torch.Size([1235192, 1])
testx.shape:  torch.Size([143348, 324])
testy.shape:  torch.Size([143348, 1])


In [53]:
train_dataset = Data.TensorDataset(trainx, trainy)
test_dataset = Data.TensorDataset(testx, testy)
batch_size = 512
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

In [54]:
class AlphaNet(nn.Module):
    def __init__(self, factor_num, fully_connect_layer_neural):
        super(AlphaNet, self).__init__()
        self.fc1_neuron = (factor_num * (factor_num - 1) + 4 * factor_num) * 3
        self.fc2_neuron = fully_connect_layer_neural
        self.fc1 = torch.nn.Linear(self.fc1_neuron, self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred

In [23]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# trainx = trainx.cuda()
# trainy = trainy.cuda()
# testx = testx.cuda()
# testy = testy.cuda()

In [55]:
alphanet = AlphaNet(feat_num, 30)
# alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.0001
loss_function = nn.MSELoss()
optimizer = optim.RMSprop(alphanet.parameters(), lr=LR, alpha=0.9)
epoch_num = 20

for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (data, label) in enumerate(train_loader):
        data = Variable(data).float()
        label = Variable(label).float()
        pred = alphanet(data)
#         print(pred.size())
#         print("epoch：", epoch, "的第" "个inputs", data.data.size(), "labels", label.data.size())
#         label = label.unsqueeze(1)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)

pred_list = []
label_list = []

for _, (data, label) in enumerate(test_loader):
    data = Variable(data).float()
    pred = alphanet(data)
    pred_list.extend(pred.tolist())
    label_list.extend(label.tolist())

# final = pd.concat([test_target, pd.DataFrame(pred_list)], axis=1)
# alpha_name = 'AlphaNetV1_Original_Input_1208'
# final.rename(columns={0: alpha_name, 'ticker': 'symbol'}, inplace=True)
# final = final.reindex(columns=['symbol', 'timestamp', alpha_name,'target'])
# final.set_index(['symbol', 'timestamp']).to_csv(output_path + "result/"+'%s_%s.csv' % (time_start, time_end))

  0%|          | 0/20 [00:00<?, ?it/s]

AlphaNet(
  (fc1): Linear(in_features=324, out_features=30, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (out): Linear(in_features=30, out_features=1, bias=True)
)


  5%|▌         | 1/20 [00:23<07:21, 23.24s/it]

Epoch:  1  loss:  0.007757011713010746


 10%|█         | 2/20 [00:46<07:03, 23.53s/it]

...

Epoch:  18  loss:  0.005051023505594282


 95%|█████████▌| 19/20 [07:36<00:24, 24.30s/it]

Epoch:  19  loss:  0.005179580140042336


100%|██████████| 20/20 [08:00<00:00, 24.05s/it]

Epoch:  20  loss:  0.005049708379171292


In [62]:
# final = pd.DataFrame(pred_list)
# final.index = index
final.reset_index(inplace=True)
alpha_name = 'AlphaNet_Original_Input_12.14'
final.rename(columns={0: alpha_name, 'ticker': 'symbol'}, inplace=True)
final

,timestamp,symbol,AlphaNet_Original_Input_12.14
0,2021-01-04 09:30:00,1,0.001154
1,2021-01-04 09:30:00,2,-0.002816
2,2021-01-04 09:30:00,4,-0.019351
3,2021-01-04 09:30:00,5,-0.011928
4,2021-01-04 09:30:00,6,0.000504
...,...,...,...
143343,2021-02-26 09:30:00,688698,0.092309
143344,2021-02-26 09:30:00,688699,0.037952
143345,2021-02-26 09:30:00,688777,0.016828
143346,2021-02-26 09:30:00,688788,0.057775


In [65]:
final.to_parquet('/home/wuwenjun/Alpha_Factor/AlphaNet_Original_Input_12.14/2021-01-01_2021-03-01.par')